# Azure Agent

## Microsoft Agent Framework를 사용한 AI 에이전트 개발 워크샵

이 워크샵에서는 Microsoft Agent Framework와 Azure OpenAI 서비스를 사용하여 다양한 AI 에이전트를 구축하는 방법을 학습합니다.

### 워크샵 목표
- Azure OpenAI 클라이언트(Chat Client, Responses Client, Assistants Client) 이해
- 함수 도구(Function Tools) 및 코드 인터프리터 사용법 학습
- 스레드 관리 및 멀티모달 기능 활용
- 실제 에이전트 구축을 통한 실습

### 필수 요구사항
- Azure OpenAI 서비스 인스턴스
- Python 3.12 이상
- 필요한 패키지 설치 (uv 사용 권장)
- Azure CLI 인증 (`az login`)

## 1. 환경 변수 설정

In [16]:
# Copyright (c) Microsoft. All rights reserved.

from dotenv import load_dotenv
import os

# 환경 변수 로드 (기존 값 덮어쓰기)
load_dotenv(override=True)

# Azure OpenAI 설정 확인
print("Azure OpenAI 설정:")
print(f"Endpoint: {os.environ.get('AZURE_OPENAI_ENDPOINT', 'Not set')}")
print(f"Chat Deployment: {os.environ.get('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME', 'Not set')}")
print(f"Responses Deployment: {os.environ.get('AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME', 'Not set')}")
print(f"API Version: {os.environ.get('AZURE_OPENAI_API_VERSION', 'Not set')}")

# Responses API를 위한 API 버전 확인 및 설정
api_version = os.environ.get('AZURE_OPENAI_API_VERSION', '2024-12-01-preview')
if api_version and api_version < '2024-12-01-preview':
    print(f"\n⚠️ 현재 API 버전({api_version})은 Responses API를 지원하지 않을 수 있습니다.")
    print("Responses API를 사용하려면 2024-12-01-preview 이상 버전이 필요합니다.")
    print("Chat Client를 사용하거나 API 버전을 업데이트하세요.")

# 필요한 환경 변수가 설정되어 있는지 확인
required_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"
]

missing_vars = [var for var in required_vars if not os.environ.get(var)]
if missing_vars:
    print(f"\n⚠️ 다음 환경 변수가 설정되지 않았습니다: {', '.join(missing_vars)}")
    print("Azure OpenAI 서비스 설정을 확인해주세요.")
else:
    print("\n✅ 모든 필수 환경 변수가 설정되었습니다.")


Azure OpenAI 설정:
Endpoint: https://aistudioaiservices343274518006.openai.azure.com
Chat Deployment: gpt-4.1
Responses Deployment: gpt-4.1
API Version: 2025-04-01-preview

✅ 모든 필수 환경 변수가 설정되었습니다.


## 2. 필수 패키지 설치

Microsoft Agent Framework 및 Azure 관련 패키지를 설치합니다.

**uv 사용 권장 (Python 3.12):**
```bash
# 프로젝트 의존성 설치
uv sync --dev

# 또는 개별 패키지 설치
uv add agent-framework agent-framework-azure-ai azure-identity python-dotenv pydantic
```

**pip 사용 (대체 방법):**

## 3. 기본 Azure OpenAI Chat Client

Azure OpenAI Chat Client를 사용하여 기본적인 에이전트를 생성하고 날씨 정보를 제공하는 함수 도구를 추가해보겠습니다.

In [7]:
import asyncio
from random import randint
from typing import Annotated

from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from pydantic import Field

# 날씨 정보를 제공하는 함수 도구
def get_weather(
    location: Annotated[str, Field(description="날씨 정보를 가져올 지역명")],
) -> str:
    """지정된 지역의 날씨 정보를 가져옵니다."""
    conditions = ["맑음", "흐림", "비", "폭풍"]
    return f"{location}의 날씨는 {conditions[randint(0, 3)]}이며, 최고 기온은 {randint(10, 30)}°C입니다."

# 비스트리밍 예제
async def non_streaming_example():
    """비스트리밍 응답 예제 (완전한 결과를 한 번에 받기)"""
    print("=== 비스트리밍 응답 예제 ===")

    # Azure Chat Client로 에이전트 생성
    agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
        instructions="당신은 도움이 되는 날씨 에이전트입니다.",
        tools=get_weather,
    )

    query = "서울의 날씨는 어떤가요?"
    print(f"사용자: {query}")
    result = await agent.run(query)
    print(f"결과: {result}\n")

# 스트리밍 예제
async def streaming_example():
    """스트리밍 응답 예제 (결과가 생성되는 대로 받기)"""
    print("=== 스트리밍 응답 예제 ===")

    # Azure Chat Client로 에이전트 생성
    agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
        instructions="당신은 도움이 되는 날씨 에이전트입니다.",
        tools=get_weather,
    )

    query = "부산의 날씨는 어떤가요?"
    print(f"사용자: {query}")
    print("에이전트: ", end="", flush=True)
    async for chunk in agent.run_stream(query):
        if chunk.text:
            print(chunk.text, end="", flush=True)
    print("\n")

# 예제 실행
await non_streaming_example()
await streaming_example()

=== 비스트리밍 응답 예제 ===
사용자: 서울의 날씨는 어떤가요?
결과: 서울의 날씨는 맑고, 최고 기온은 12도입니다. Outdoor 활동하기 좋은 날씨입니다!

=== 스트리밍 응답 예제 ===
사용자: 부산의 날씨는 어떤가요?
에이전트: 부산의 날씨는 맑고, 최고 기온은 30°C입니다. 산책이나 야외 활동하기 좋은 날씨입니다.



## 4. Azure OpenAI Chat Client - 함수 도구 패턴

함수 도구를 다양한 방식으로 구성하는 방법을 학습해보겠습니다.

In [8]:
from datetime import datetime, timezone
from agent_framework import ChatAgent

# 현재 시간을 제공하는 함수 도구
def get_time() -> str:
    """현재 UTC 시간을 가져옵니다."""
    current_time = datetime.now(timezone.utc)
    return f"현재 UTC 시간은 {current_time.strftime('%Y-%m-%d %H:%M:%S')}입니다."

# 1. 에이전트 수준에서 도구 정의
async def tools_on_agent_level():
    """에이전트 생성 시 도구를 정의하는 예제"""
    print("=== 에이전트 수준에서 도구 정의 ===")

    # 에이전트 생성 시 도구 제공
    agent = ChatAgent(
        chat_client=AzureOpenAIChatClient(credential=AzureCliCredential()),
        instructions="당신은 날씨와 시간 정보를 제공할 수 있는 도움이 되는 어시스턴트입니다.",
        tools=[get_weather, get_time],  # 에이전트 생성 시 도구 정의
    )

    # 첫 번째 쿼리 - 에이전트가 날씨 도구 사용
    query1 = "뉴욕의 날씨는 어떤가요?"
    print(f"사용자: {query1}")
    result1 = await agent.run(query1)
    print(f"에이전트: {result1}\n")

    # 두 번째 쿼리 - 에이전트가 시간 도구 사용
    query2 = "현재 UTC 시간은 몇 시인가요?"
    print(f"사용자: {query2}")
    result2 = await agent.run(query2)
    print(f"에이전트: {result2}\n")

# 2. 런 메서드에서 도구 전달
async def tools_on_run_level():
    """런 메서드에서 도구를 전달하는 예제"""
    print("=== 런 메서드에서 도구 전달 ===")

    # 도구 없이 에이전트 생성
    agent = ChatAgent(
        chat_client=AzureOpenAIChatClient(credential=AzureCliCredential()),
        instructions="당신은 도움이 되는 어시스턴트입니다.",
    )

    # 날씨 도구와 함께 첫 번째 쿼리
    query1 = "시애틀의 날씨는 어떤가요?"
    print(f"사용자: {query1}")
    result1 = await agent.run(query1, tools=[get_weather])
    print(f"에이전트: {result1}\n")

    # 시간 도구와 함께 두 번째 쿼리
    query2 = "현재 UTC 시간은 몇 시인가요?"
    print(f"사용자: {query2}")
    result2 = await agent.run(query2, tools=[get_time])
    print(f"에이전트: {result2}\n")

# 예제 실행
await tools_on_agent_level()
await tools_on_run_level()

=== 에이전트 수준에서 도구 정의 ===
사용자: 뉴욕의 날씨는 어떤가요?
에이전트: 뉴욕의 현재 날씨는 비가 내리고 있으며, 최고 기온은 11°C입니다. 외출 시 우산을 준비하시는 것이 좋겠습니다.

사용자: 현재 UTC 시간은 몇 시인가요?
에이전트: 현재 UTC 시간은 2025년 12월 15일 08시 54분 11초입니다.

=== 런 메서드에서 도구 전달 ===
사용자: 시애틀의 날씨는 어떤가요?
에이전트: 시애틀의 현재 날씨는 흐림이고, 최고 기온은 10°C입니다.

사용자: 현재 UTC 시간은 몇 시인가요?
에이전트: 현재 UTC 시간은 2025년 12월 15일 08시 54분 15초입니다.



## 5. Azure OpenAI Chat Client - 스레드 관리

스레드를 사용하여 대화 컨텍스트를 유지하는 방법을 학습해보겠습니다.

In [9]:
from agent_framework import AgentThread

# 1. 자동 스레드 생성 예제
async def automatic_thread_example():
    """자동 스레드 생성 예제"""
    print("=== 자동 스레드 생성 예제 ===")

    agent = ChatAgent(
        chat_client=AzureOpenAIChatClient(credential=AzureCliCredential()),
        instructions="당신은 도움이 되는 날씨 에이전트입니다.",
        tools=get_weather,
    )

    # 첫 번째 대화 - 스레드가 제공되지 않으면 자동으로 생성됨
    query1 = "서울의 날씨는 어떤가요?"
    print(f"사용자: {query1}")
    result1 = await agent.run(query1)
    print(f"에이전트: {result1.text}")

    # 두 번째 대화 - 여전히 스레드가 제공되지 않으면 새로운 스레드 생성
    query2 = "제가 마지막으로 물어본 도시는 어디인가요?"
    print(f"\n사용자: {query2}")
    result2 = await agent.run(query2)
    print(f"에이전트: {result2.text}")
    print("참고: 각 호출은 별도의 스레드를 생성하므로 에이전트가 이전 컨텍스트를 기억하지 못합니다.\n")

# 2. 스레드 지속성 예제
async def thread_persistence_example():
    """스레드 지속성을 사용한 컨텍스트 유지 예제"""
    print("=== 스레드 지속성 예제 ===")
    print("동일한 스레드를 여러 대화에 사용하여 컨텍스트를 유지합니다.\n")

    agent = ChatAgent(
        chat_client=AzureOpenAIChatClient(credential=AzureCliCredential()),
        instructions="당신은 도움이 되는 날씨 에이전트입니다.",
        tools=get_weather,
    )

    # 재사용할 새 스레드 생성
    thread = agent.get_new_thread()

    # 첫 번째 대화
    query1 = "도쿄의 날씨는 어떤가요?"
    print(f"사용자: {query1}")
    result1 = await agent.run(query1, thread=thread)
    print(f"에이전트: {result1.text}")

    # 동일한 스레드를 사용한 두 번째 대화 - 컨텍스트 유지
    query2 = "런던은 어떤가요?"
    print(f"\n사용자: {query2}")
    result2 = await agent.run(query2, thread=thread)
    print(f"에이전트: {result2.text}")

    # 세 번째 대화 - 에이전트가 이전 도시들을 기억해야 함
    query3 = "제가 물어본 도시들 중 어느 곳의 날씨가 더 좋나요?"
    print(f"\n사용자: {query3}")
    result3 = await agent.run(query3, thread=thread)
    print(f"에이전트: {result3.text}")
    print("참고: 에이전트가 동일한 스레드의 이전 메시지를 기억합니다.\n")

# 예제 실행
await automatic_thread_example()
await thread_persistence_example()

=== 자동 스레드 생성 예제 ===
사용자: 서울의 날씨는 어떤가요?
에이전트: 서울의 현재 날씨는 폭풍이며, 최고 기온은 16°C입니다. 외출 시 안전에 유의하시고 우산이나 방수 용품을 준비하시길 권장합니다.

사용자: 제가 마지막으로 물어본 도시는 어디인가요?
에이전트: 직전 대화에서는 특정 도시 이름을 언급하지 않으셨습니다. 확인하고 싶은 도시가 있으시면 말씀해 주세요. 해당 도시의 날씨 정보를 알려드릴 수 있습니다.
참고: 각 호출은 별도의 스레드를 생성하므로 에이전트가 이전 컨텍스트를 기억하지 못합니다.

=== 스레드 지속성 예제 ===
동일한 스레드를 여러 대화에 사용하여 컨텍스트를 유지합니다.

사용자: 도쿄의 날씨는 어떤가요?
에이전트: 도쿄의 현재 날씨는 흐림이며, 최고 기온은 22°C입니다.  
외출 시 우산을 챙기거나 기온을 참고해 주세요!

사용자: 런던은 어떤가요?
에이전트: 런던의 현재 날씨는 흐림이고, 최고 기온은 21°C입니다.  
외출 시 참고해 주세요!

사용자: 제가 물어본 도시들 중 어느 곳의 날씨가 더 좋나요?
에이전트: 도쿄와 런던 모두 현재 흐린 날씨입니다.  
기온을 기준으로 보면 도쿄는 22°C, 런던은 21°C로 두 도시 모두 온화한 편이지만 도쿄가 약간 더 따뜻합니다.

날씨가 "좋다"는 기준이 개인에 따라 다를 수 있지만, 흐림 상태에서는 따뜻한 도쿄가 좀 더 쾌적할 수 있습니다.  
단, 비나 습도 정보는 제공되지 않아 자세한 평가는 어려운 점 참고해 주세요!
참고: 에이전트가 동일한 스레드의 이전 메시지를 기억합니다.



## 6. 워크샵 실습: 나만의 에이전트 만들기

지금까지 학습한 내용을 바탕으로 여러 기능을 결합한 종합적인 에이전트를 만들어보겠습니다.

In [21]:
# 실습 과제: 종합 에이전트 만들기
# 다음 기능들을 가진 "여행 계획 도우미" 에이전트를 만들어보세요:
# 1. 날씨 정보 제공 (get_weather 함수 사용)
# 2. 현재 시간 제공 (get_time 함수 사용)
# 3. 여행 예산 계산 (새로운 함수 작성)
# 4. 스레드를 사용한 대화 컨텍스트 유지

# 여행 예산을 계산하는 함수
def calculate_travel_budget(
    days: Annotated[int, Field(description="여행 일수")],
    daily_budget: Annotated[float, Field(description="일일 예산 (원)")],
    num_people: Annotated[int, Field(description="여행 인원 수")] = 1,
) -> str:
    """여행 예산을 계산합니다."""
    total_budget = days * daily_budget * num_people
    return f"{num_people}명이 {days}일간 여행할 경우, 총 예산은 {total_budget:,.0f}원입니다."

# 항공권 가격 추정 함수
def estimate_flight_price(
    destination: Annotated[str, Field(description="목적지")],
    departure: Annotated[str, Field(description="출발지")] = "서울",
) -> str:
    """항공권 가격을 추정합니다."""
    prices = {
        "도쿄": 300000,
        "오사카": 280000,
        "방콕": 450000,
        "싱가포르": 550000,
        "파리": 1200000,
        "런던": 1300000,
        "뉴욕": 1400000,
        "시드니": 1100000,
    }
    price = prices.get(destination, 500000)
    return f"{departure}에서 {destination}까지의 왕복 항공권 예상 가격은 약 {price:,.0f}원입니다."

# 여행 계획 도우미 에이전트 생성
async def travel_planning_agent():
    """종합적인 여행 계획 도우미 에이전트"""
    print("=== 여행 계획 도우미 에이전트 ===\n")

    # 모든 도구를 포함한 에이전트 생성
    async with ChatAgent(
        chat_client=AzureOpenAIChatClient(credential=AzureCliCredential()),
        instructions="""당신은 여행 계획을 도와주는 전문 에이전트입니다. 
        다음과 같은 기능을 제공할 수 있습니다:
        1. 목적지의 날씨 정보
        2. 현재 시간 확인
        3. 여행 예산 계산
        4. 항공권 가격 추정
        
        사용자의 질문에 친절하고 자세하게 답변해주세요.""",
        tools=[get_weather, get_time, calculate_travel_budget, estimate_flight_price],
    ) as agent:
        # 대화 컨텍스트를 유지하기 위한 스레드 생성
        thread = agent.get_new_thread()

        # 시나리오 1: 기본 정보 수집
        query1 = "도쿄로 여행을 가려고 하는데, 현재 날씨는 어떤가요?"
        print(f"사용자: {query1}")
        result1 = await agent.run(query1, thread=thread)
        print(f"에이전트: {result1.text}\n")

        # 시나리오 2: 항공권 정보
        query2 = "도쿄까지 항공권은 얼마나 할까요?"
        print(f"사용자: {query2}")
        result2 = await agent.run(query2, thread=thread)
        print(f"에이전트: {result2.text}\n")

        # 시나리오 3: 예산 계산
        query3 = "5일 동안 2명이 가는데, 일일 예산을 15만원으로 잡으면 총 얼마나 필요한가요?"
        print(f"사용자: {query3}")
        result3 = await agent.run(query3, thread=thread)
        print(f"에이전트: {result3.text}\n")

        # 시나리오 4: 종합 상담 (컨텍스트 활용)
        query4 = "지금까지 이야기한 내용을 바탕으로 총 여행 경비를 정리해주세요."
        print(f"사용자: {query4}")
        result4 = await agent.run(query4, thread=thread)
        print(f"에이전트: {result4.text}\n")

        # 시나리오 5: 현재 시간 확인
        query5 = "참고로 현재 시간은 몇 시인가요?"
        print(f"사용자: {query5}")
        result5 = await agent.run(query5, thread=thread)
        print(f"에이전트: {result5.text}\n")

# 예제 실행
await travel_planning_agent()

print("\n" + "="*60)
print("워크샵을 완료했습니다! 🎉")
print("="*60)
print("\n다음 단계:")
print("1. 위 코드를 수정하여 다른 도구들을 추가해보세요")
print("2. 다른 클라이언트(Chat Client, Responses Client)로 변경해보세요")
print("3. 코드 인터프리터를 추가하여 데이터 분석 기능을 넣어보세요")
print("4. 이미지 분석 기능을 추가하여 여행지 사진을 분석해보세요")

=== 여행 계획 도우미 에이전트 ===

사용자: 도쿄로 여행을 가려고 하는데, 현재 날씨는 어떤가요?
에이전트: 도쿄의 현재 날씨는 비가 내리고 있습니다. 최고 기온은 20°C로, 다소 서늘한 편입니다.

우산이나 우비를 챙기고, 기온에 맞춰 겉옷을 준비하시면 좋은 여행이 되실 거예요! 다른 정보가 필요하신가요?

사용자: 도쿄까지 항공권은 얼마나 할까요?
에이전트: 서울에서 도쿄까지의 왕복 항공권 예상 가격은 약 300,000원입니다.

항공권 가격은 여행 시기, 예약 시점, 항공사에 따라 달라질 수 있으니 참고해 주세요! 추가로 일정이나 여행 예산 계산이 필요하시다면 말씀해 주세요.

사용자: 5일 동안 2명이 가는데, 일일 예산을 15만원으로 잡으면 총 얼마나 필요한가요?
에이전트: 2명이 5일간 도쿄 여행을 하면서, 일일 예산을 15만 원으로 잡으면 총 필요한 여행 경비는 1,500,000원입니다.

여기에 항공권 비용(약 300,000원 x 2명 = 600,000원)을 더하면 전체 예상 경비는 2,100,000원이 됩니다.

여행 일정이나 추가 정보가 필요하시면 언제든 말씀해 주세요!

사용자: 지금까지 이야기한 내용을 바탕으로 총 여행 경비를 정리해주세요.
에이전트: 지금까지 안내드린 내용을 바탕으로 도쿄 5일 여행 경비를 정리해드리겠습니다.

1. 항공권 비용  
- 왕복 항공권 예상 가격: 약 300,000원 × 2명 = 600,000원

2. 여행 예산  
- 일일 예산: 150,000원  
- 5일 × 2명 = 1,500,000원

3. 총 예상 여행 경비  
- 항공권 + 여행 예산 = 600,000원 + 1,500,000원 = 2,100,000원

즉, 2명이 5일간 도쿄로 여행을 떠날 경우 준비해야 하는 총 경비는 약 2,100,000원입니다.

혹시 숙소, 교통, 관광지 입장료 등 세부 항목이 궁금하시거나, 여행 일정에 대해 더 자세한 안내가 필요하시면 말씀해 주세요!

사용자: 참고로 현재 시간은 몇 시인가요?
에이전트: 현재 U

## 추가 학습 자료 및 참고사항

### 주요 개념 정리

1. **Azure OpenAI 클라이언트 유형**
   - **Chat Client**: 직접 채팅 기반 상호작용
   - **Responses Client**: 구조화된 응답 생성
   - **Assistants Client**: 영구 어시스턴트 및 자동 생명주기 관리

2. **함수 도구 (Function Tools)**
   - 에이전트 생성 시 정의: 모든 쿼리에서 사용 가능
   - 런 메서드에서 전달: 특정 쿼리에만 사용
   - 혼합 방식: 기본 도구 + 추가 도구

3. **스레드 관리**
   - 자동 스레드: 각 쿼리마다 새로운 컨텍스트
   - 명시적 스레드: 대화 컨텍스트 유지
   - 서비스 스레드 ID: 서버 측 대화 이력 저장

4. **코드 인터프리터**
   - Python 코드 실행 (Python 3.12+ 지원)
   - 수학적 계산 및 데이터 분석
   - 파일 업로드 및 처리

5. **멀티모달 기능**
   - 이미지 분석
   - 텍스트와 이미지 결합
   - 비전 모델 활용

### Python 3.12 환경 설정

```bash
# uv로 Python 3.12 환경 생성
uv venv --python 3.12

# 프로젝트 의존성 설치
uv sync --dev

# Python 버전 확인
uv run python --version  # Python 3.12.x

# Jupyter 노트북 실행
uv run jupyter notebook
```

### 환경 변수 설정 예제

```bash
# .env 파일
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME=gpt-4
AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME=gpt-4
AZURE_OPENAI_API_VERSION=2024-05-01-preview
```

### 인증 방법

1. **Azure CLI 인증** (권장)
   ```bash
   az login
   ```

2. **API 키 인증**
   ```python
   from openai import AsyncAzureOpenAI
   client = AsyncAzureOpenAI(api_key="your-api-key", ...)
   ```

3. **Managed Identity**
   ```python
   from azure.identity import DefaultAzureCredential
   credential = DefaultAzureCredential()
   ```

### 추가 리소스

- Microsoft Agent Framework 문서
- Azure OpenAI 서비스 문서
- GitHub 샘플 코드 저장소
- Python 3.12 공식 문서

### 다음 단계

1. 프로덕션 환경을 위한 에러 핸들링 추가
2. 로깅 및 모니터링 구현
3. 성능 최적화 및 캐싱 전략
4. 보안 모범 사례 적용
5. Python 3.12의 새로운 기능 활용
   - 개선된 타입 힌트
   - f-string 성능 향상
   - 새로운 pathlib 기능